In [2]:
from deeptrace import *
stack = BrainStack(channel_folders=['../../sampledata/210723_NAc326F_488_s3_0_8x_13-31-25/',
                   '../../sampledata/210723_NAc326F_640_s3_0_8x_11-50-51/'])
trailmap_channel = 1
stack.set_active_channels([trailmap_channel])


In [3]:

files = stack.channel_files[trailmap_channel]
from tqdm.notebook import tqdm
pbar = tqdm()
models = trailmap_list_models()
model_path = models[4]
trailmap_segment_tif_files(model_path, files,pbar = pbar)    

0it [00:00, ?it/s]

Loading TRAILMAP network and model.
Using model weight denardo_trapmodel4.hdf5
